We will query Gaia DR3 to find various features of stars with specific criteria

In [36]:
from astroquery.gaia import Gaia
from astropy.table import Column
import pandas as pd

table="gaiadr3.gaia_source"
Gaia.load_table(table)


query = """select top 10 
source_id, ra, ra_error, dec, dec_error, parallax, parallax_error, pmra, pmra_error, pmdec, pmdec_error
from gaiadr3.gaia_source 

where pmra is not null
and pmdec is not null
and radial_velocity is not null

and pmra !=0
and pmdec !=0
and radial_velocity !=0

and parallax > 0
"""

results=Gaia.launch_job_async(query).get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [37]:
df=results.to_pandas()
print(df)

             source_id          ra  ra_error        dec  dec_error  parallax  \
0  4052307003895216512  275.430005  0.032665 -27.162098   0.029489  0.124304   
1  3001468183198140800   94.630312  0.023736 -10.632227   0.025125  4.520103   
2  4313545645245332864  286.139480  0.023021  12.043162   0.025816  0.783336   
3  4255077877498765696  281.903655  0.032070  -5.076782   0.029423  0.994220   
4   670746696180987648  121.677030  0.023523  21.126505   0.016885  1.002062   
5  4096420376877032832  275.160703  0.026095 -17.986049   0.023769  0.428026   
6  3416522621408031360   81.633791  0.029062  24.311811   0.021479  0.800101   
7  6144317404055927168  180.062053  0.705305 -45.748970   0.583749  5.916363   
8   460980974485934976   43.508028  0.024635  58.383391   0.028530  7.652620   
9  5800518607914795648  224.521525  0.141310 -67.540155   0.199139  1.275412   

   parallax_error       pmra  pmra_error      pmdec  pmdec_error  
0        0.041029  -3.419536    0.043168  -6.489931 

We will calculate for these stars the distances in parsec, and the relative error using the parallax method, the parallax angle is given in milliarcseconds.
$$d=1000/p$$ 
where $d$ is the distance in parsec and $p$ is the parallax angle in milliarcseconds.
The error in the distance is given by the formula:
$$\Delta d = \frac{1000 \Delta p}{p^2}$$

where $\Delta p$ is the error in the parallax angle in milliarcseconds.

In [38]:
distance=Column(1000/df["parallax"],name="distance")
distance_error = Column(1000 * df["parallax_error"] / (df["parallax"] ** 2),name="distance_error")
results.add_columns([distance, distance_error])

In [39]:
df=results.to_pandas()
print(df)

             source_id          ra  ra_error        dec  dec_error  parallax  \
0  4052307003895216512  275.430005  0.032665 -27.162098   0.029489  0.124304   
1  3001468183198140800   94.630312  0.023736 -10.632227   0.025125  4.520103   
2  4313545645245332864  286.139480  0.023021  12.043162   0.025816  0.783336   
3  4255077877498765696  281.903655  0.032070  -5.076782   0.029423  0.994220   
4   670746696180987648  121.677030  0.023523  21.126505   0.016885  1.002062   
5  4096420376877032832  275.160703  0.026095 -17.986049   0.023769  0.428026   
6  3416522621408031360   81.633791  0.029062  24.311811   0.021479  0.800101   
7  6144317404055927168  180.062053  0.705305 -45.748970   0.583749  5.916363   
8   460980974485934976   43.508028  0.024635  58.383391   0.028530  7.652620   
9  5800518607914795648  224.521525  0.141310 -67.540155   0.199139  1.275412   

   parallax_error       pmra  pmra_error      pmdec  pmdec_error     distance  \
0        0.041029  -3.419536    0.0431